In [13]:
using JuMP, GLPK
using DataFrames, CSV

In [14]:
# PARAMETERS
F = 3       # number of fish types
G = 3       # number of fishing grounds
I = 4       # number of processing facilities
J = 100     # number of cities

# DATA: FISH 
fish_price = [201.35, 156.53, 265, 380]


# DATA: FISHING GROUND 
df_fishingground_max = DataFrame(
    Milkfish = [680.94, 3290.55, 3732.93],
    Tilapia = [34.91, 9776.8, 5649.49],
    Shrimp = [0, 2155.94, 5354.92]
)
display(df_fishingground_max)

# DATA: PROCESSING FACILITY
processing_cap = [648000, 127750, 145600, 27000 ]

# DATA: CITIES
budget = [4548.063, 4198.212, 4198.212, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 4198.212, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 4198.212, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 4198.212, 4198.212, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51, 3498.51, 3498.51, 3498.51, 3498.51, 4198.212, 4198.212, 3498.51, 4198.212, 4198.212, 4198.212, 3498.51, 3498.51, 3498.51, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51, 4198.212, 3498.51, 3498.51]
population = [24922000, 2960048, 2100000, 1776949, 1661584, 1273219, 964169, 887399, 886722, 803159, 728402, 714978, 703141, 697315, 689992, 664625, 654220, 651813, 629616, 606293, 600783, 574000, 543445, 539671, 497604, 496794, 462928, 456059, 450583, 443954, 440656, 425758, 414812, 407437, 388305, 386451, 385398, 380522, 376933, 372931, 372910, 366358, 364116, 363115, 355330, 354666, 351437, 342769, 327325, 326001, 325079, 313631, 312116, 307079, 296202, 295644, 293244, 289820, 285348, 278924, 273306, 263048, 261189, 260317, 254453, 251881, 247543, 230998, 225673, 218500, 216546, 210503, 210452, 209533, 209230, 207314, 207010, 205424, 203598, 200198, 195398, 193936, 191210, 190712, 188376, 186960, 182237, 179292, 174302, 173502, 173403, 172605, 172433, 172073, 171107, 169953, 168470, 166334, 161868, 160987]

# DATA FOR OBJECTIVE FUNCTION
health_value = [263.53, 261.02, 239.86, 291.76]
df_fishing_cost = DataFrame(CSV.File("fishing_cost.csv"))
df_transportation_cost = DataFrame(CSV.File("transportation_cost.csv"))
println("Fishing Cost")
display(df_fishing_cost)
println("Transportation Cost")
display(df_transportation_cost)

operational_cost = [654, 1003, 70, 470]

Row,Milkfish,Tilapia,Shrimp
,Float64,Float64,Float64
1,680.94,34.91,0.0
2,3290.55,9776.8,2155.94
3,3732.93,5649.49,5354.92


Fishing Cost


Row,1,2,3,4
,Float64,Float64,Float64,Float64
1,1064.77,1064.77,1064.77,1064.77
2,378.894,378.894,378.894,378.894
3,343.741,343.741,343.741,343.741


Transportation Cost


Row,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05,1055.05
2,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664,10.3664
3,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537,23.8537
4,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657,12.6657

4-element Vector{Int64}:
  654
 1003
   70
  470

In [15]:
# VARIABLES
m = Model(GLPK.Optimizer)
@variable(m, x[1:F, 1:G, 1:I])
@variable(m, y[1:F, 1:I, 1:J])
@variable(m, z[1:I], Bin)
@variable(m, s[1:J])

100-element Vector{VariableRef}:
 s[1]
 s[2]
 s[3]
 s[4]
 s[5]
 s[6]
 s[7]
 s[8]
 s[9]
 s[10]
 s[11]
 s[12]
 s[13]
 ⋮
 s[89]
 s[90]
 s[91]
 s[92]
 s[93]
 s[94]
 s[95]
 s[96]
 s[97]
 s[98]
 s[99]
 s[100]

In [32]:
# CONSTRAINTS
# Fishing Restriction Constraint
for f in 1:F
    for g in 1:G
        @constraint(m, sum(x[f, g, i] for i in 1:I) <= df_fishingground_max[g, f])
    end
end

# Distribution/Processing Capacity Constraint
for i in 1:I
    @constraint(m, sum(y[f, i, j] for f in 1:F for j in 1:J) <= processing_cap[i])
end

# Min/Max Fish Consumption in each City 
for j in 1:J # arbitrary number for now
    @constraint(m, sum(y[f, i, j] for f in 1:F for i in 1:I) <= 100000000) 
    @constraint(m, sum(y[f, i, j] for f in 1:F for i in 1:I) >= 100)
end
 
# Total Fish Distributed < Total Fish Caught
@constraint(m, sum(y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) <= sum(x[f, g, i] for f in 1:F for g in 1:G for i in 1:I))

# Budget Constraint
for j in 1:J
    @constraint(m, sum(y[f, i, j]*fish_price[f] for f in 1:F for i in 1:I)  <= population[j]*budget[j] + s[j])
end

# Indicator variables
# If we close facility i, we cannot send any fish to facility i
for f in 1:F
    for g in 1:G
        for i in 1:I
            @constraint(m, x[f,g,i] <= z[i] * df_fishingground_max[g, f])
        end
    end
end

# Indicator variables
# If we close facility i, we cannot send any fish from facility i
for f in 1:F
    for i in 1:I
        for j in 1:J
            @constraint(m, y[f,i,j] <= z[i] * processing_cap[i])
        end
    end
end

# Non-negativity constraints
for f in 1:F
    for g in 1:G
        for i in 1:I
            @constraint(m, x[f, g, i] >= 0)
        end
    end
    for i in 1:I
        for j in 1:J
            @constraint(m, y[f, i, j] >= 0)
        end
    end
end
for j in 1:J
    @constraint(m, s[j] >= 0)
end      



In [33]:
# OBJECTIVE FUNCTION
@objective(m, Max, sum(health_value[f]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(df_fishing_cost[g, i]*x[f, g, i] for f in 1:F for g in 1:G for i in 1:I) - sum(df_transportation_cost[i,j]*y[f, i, j] for f in 1:F for i in 1:I for j in 1:J) - sum(operational_cost[i]*z[i] for i in 1:I) - sum(s[j] for j in 1:J))

-791.5179347578446 y[1,1,1] - 791.5179347578446 y[1,1,2] - 791.5179347578446 y[1,1,3] - 791.5179347578446 y[1,1,4] - 791.5179347578446 y[1,1,5] - 791.5179347578446 y[1,1,6] - 791.5179347578446 y[1,1,7] - 791.5179347578446 y[1,1,8] - 791.5179347578446 y[1,1,9] - 791.5179347578446 y[1,1,10] - 791.5179347578446 y[1,1,11] - 791.5179347578446 y[1,1,12] - 791.5179347578446 y[1,1,13] - 791.5179347578446 y[1,1,14] - 791.5179347578446 y[1,1,15] - 791.5179347578446 y[1,1,16] - 791.5179347578446 y[1,1,17] - 791.5179347578446 y[1,1,18] - 791.5179347578446 y[1,1,19] - 791.5179347578446 y[1,1,20] - 791.5179347578446 y[1,1,21] - 791.5179347578446 y[1,1,22] - 791.5179347578446 y[1,1,23] - 791.5179347578446 y[1,1,24] - 791.5179347578446 y[1,1,25] - 791.5179347578446 y[1,1,26] - 791.5179347578446 y[1,1,27] - 791.5179347578446 y[1,1,28] - 791.5179347578446 y[1,1,29] - 791.5179347578446 y[1,1,30] - [[...1280 terms omitted...]] - s[71] - s[72] - s[73] - s[74] - s[75] - s[76] - s[77] - s[78] - s[79] - s[80]

In [34]:
optimize!(m)

In [35]:
solution_summary(m)

* Solver : GLPK

* Status
  Result count       : 0
  Termination status : INFEASIBLE
  Message from the solver:
  "Unable to start the search, because LP relaxation of the MIP problem instance has no primal feasible solution. (This code may appear only if the presolver is enabled.)"

* Candidate solution (result #1)
  Primal status      : NO_SOLUTION
  Dual status        : NO_SOLUTION
  Objective bound    : 0.00000e+00
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 2.10000e-01


In [31]:
println(value.(x))

[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
